# Time series comparison between FVCOM and MPOS
**Author: Jun Sasaki, Coded on January 7, 2025, Updated on February 7, 2025 ([MIT License](https://opensource.org/licenses/MIT))**<br>
Compare FVCOM output netcdf and Mpos data at a specified node/nele and siglay.
- Suppose Mpos netCDF and FVCOM output netCDF are prepared.
- FVCOM variables are defined at siglay. The depth at a specified siglay will change with time because of the change in the surface level.
- Interpolate Mpos item values at the specified siglay in FVCOM to precisely compare them.

## Methods
1. Load the FVCOM output netcdf file into xfvcom, specify the node or nele value for the horizontal coordinate in the scalar and the siglay or siglev values for the vertical coordinate in the list, extract the corresponding parts, and plot them.
2. Load the Mpos observation data and extract the data closest to the datetime and depth corresponding to 1. In doing so, set the allowable error and extract the data.
3. Extract the common parts of 1 and 2.
4. Use 3 to evaluate the accuracy of the calculation results.

In [ ]:
from xfvcom import FvcomDataLoader, FvcomAnalyzer, FvcomPlotConfig, FvcomPlotter
from xfvcom.helpers import FrameGenerator
from xfvcom.helpers_utils import evaluate_model_scores, generate_test_data
from xmpos import MposDataLoader, MposAnalyzer, MposPlotConfig, MposPlotter
import os
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import hvplot.xarray
import panel.widgets as pnw
import holoviews as hv
## Global parameters
import cartopy.crs as ccrs

width=800; height=200

### Load FVCOM output netcdf into xarray.Dataset

In [ ]:
output_dir = "./"
output_dir = os.path.expanduser(output_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
base_path = "~/Github/TB-FVCOM/goto2023"
base_path = os.path.expanduser(base_path)
obcfile_path =  f"{base_path}/input/TokyoBay18_obc.dat"
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
index_ncfile=0
ncfile_path = f"{base_path}/output/{ncfiles[index_ncfile]}"
fvcom = FvcomDataLoader(ncfile_path=ncfile_path, obcfile_path=obcfile_path, time_tolerance=5)
ds_fvcom = fvcom.ds
ds_fvcom

### Select a specified `node` of MPOS
Note: `nele` is unsupported.

In [ ]:
bl_stns = [(140.023333, 35.610833),  # 01kemigawa
           (139.833889, 35.490278),  # 02kawasaki
           (139.941667, 35.640000),  # 03urayasu
           (139.954167, 35.536944)]  # 04chiba1gou
stns = ["01kemigawa", "02kawasaki", "03urayasu", "04chiba1gou"]
index_stn = 1
stn = stns[index_stn]
lon, lat = bl_stns[index_stn]
f_a = FvcomAnalyzer(ds_fvcom)
node = f_a.nearest_neighbor(lon, lat, node=True, distances=False)
print(f"node={node} is the nearest node for ({lon}, {lat})")
# Confirm
print(f"The exact (lon, lat) = ({ds_fvcom.lon[node].values.item()}, {ds_fvcom.lat[node].values.item()} for node={node}")

### Plot the node in a spatial map.

In [ ]:
fvcom_plotter = FvcomPlotter(fvcom.ds, FvcomPlotConfig(figsize=(6,8)))
# Plot the mesh colored by bathymetry
#fig, ax = plt.subplots(1,1, figsize=(6, 8))
#ax = fvcom_plotter.plot_mesh(color_by="h", figsize=(8,10))
# Plot the mesh as a wireframe
#ax = fvcom_plotter.plot_mesh(color_by=None, ax=ax)
levels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 50, 100, 200, 300, 400, 500]
ax=fvcom_plotter.plot_2d(da=ds_fvcom.h, cmap='jet', with_mesh=True, coastlines=True, obclines=True, add_tiles=True,
                         title=None, levels=levels, save_path="mesh_latlon_fixed.png")
fvcom_plotter.add_marker(ax=ax, x=lon, y=lat, marker='x')
fvcom_plotter.plot_2d(da=ds_fvcom.h, with_mesh=True, use_latlon=False, cmap='jet', levels=20, save_path="mesh_latlon_fixed.png")
fvcom_plotter.plot_2d(da=ds_fvcom.temp[20,0,:], cmap='jet', levels=20, with_mesh=True, coastlines=True, obclines=True,
                      plot_grid=True, save_path="mesh_latlon_fixed.png")

### Plot with `FrameGenerator` to handle dynamic `post_process_func`

In [ ]:
def dynamic_custom_plot(ax, da, time):
    """
    An example of post_process_func for plotting datetime.
    Note: ax can be accessed without special treatment.
          However, to access da and time, a wrapper of FrameGenerator is necessary.

    Parameters:
    - ax: matplotib axis.
    - da: DataArray.
    - time: Frame time.    
    """
    datetime = pd.Timestamp(da.time.item()).strftime('%Y-%m-%d %H:%M:%S')
    ax.set_title(f"Time: {datetime}")

# Specify var_name and siglay if any
var_name = "temp"
time = 40
siglay = 0
da = ds_fvcom[var_name][:,siglay,:] # time must be included in da as reusing a tool for animation.
time_str = ds_fvcom.time.isel(time=time).values
time_str = pd.to_datetime(time_str).strftime("%Y%m%d")
# Set plot_kwargs for `ax.tricontourf(**kwargs)`.
plot_kwargs={"verbose": False, "vmin": 9, "vmax": 20,
             "levels": [9, 9.2, 9.4, 9.6, 9.8, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 15, 16, 17, 18, 19, 20],
             "cmap":"jet"}
#plot_kwargs={"verbose": False, "vmin": 10, "vmax": 20, "levels": 20, "cmap": "jet"}
#plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet", "with_mesh": True, "plot_grid": True, "add_tiles": True}
#plot_kwargs={}
save_path = os.path.join(output_dir, f"{var_name}_{time_str}.png")

FrameGenerator.plot_data(data_array=da, time=time, plotter=fvcom_plotter,
                              save_path=save_path, post_process_func=dynamic_custom_plot,
                              plot_kwargs=plot_kwargs)
#fig=ax.figure
#fig.savefig(save_path, 'bbox_inches'='tight', dpi=300)
#fig.savefig(save_path, bbox_inches='tight', dpi=300)

Multiple siglays can be plotted by defining a list of `siglays` and `colors`.

## Load Mpos.

In [ ]:
#base_path = '~/Github/xmpos/data/MLIT_edited_Mpos/nc_structured/'
base_path = f"{os.environ['DATA_DIR']}/MLIT_edited_Mpos/nc_structured/"
loader = MposDataLoader(base_path=base_path)
ds_mpos = loader.load_structured_nc(stn=stn, start=2015, end=2020, dask=False)
#print(ds_mpos.temp[30000:,5].values)
#print(ds_mpos.z[5].item())

### Plot time-series of vertical profile of a `varname`.
- Note: `clim=(cmin, cmax)` must be specified while `ylim=None` is acceptable. Add `clim=None` in the future. 

In [ ]:
var_name = 'temp'
time_slice=(pd.Timestamp('2020-01-01'), pd.Timestamp('2020-01-15'))
mpos_config = MposPlotConfig(figsize=(8, 2), dpi=72, cmap="jet", levels=21)
mpos_plotter = MposPlotter(ds_mpos, config=mpos_config)
# Create the contour plot
fig, ax, cbar = mpos_plotter.plot_contour(
    ds_mpos[var_name], xlim=('2015-01-01', '2021-02-01'), ylim=None, clim=(10,14),
    stn=stn, rolling=False, window=25
)
# Adjust the x-axis and add more annotations
#ax.set_xlim(pd.Timestamp('2020-01-01'), pd.Timestamp('2020-01-15'))
ax.set_xlim(time_slice)
#ax.set_title("Adjusted DO at Kemigawa")
plt.show()

### Time-series interactive plot at each z 

In [ ]:
z_slider =  pnw.IntSlider(name='z', start=0, end=len(ds_mpos.z) - 1, value=0)
ds_mpos[var_name].sel(time=slice(time_slice[0], time_slice[1])).interactive.isel(z=z_slider).hvplot().opts(width=700, height=height)

## Extract target FVCOM time series depths `z_dfs` at the specified `node` and `siglay`
The depths from the surface `zdfs` should be selected referring to MPOS time series above.

### Slice `ds_fvcom[var_name]` at specified siglays and plot its time-series.
First, quick plot interactively.

In [ ]:
#var_name = 'temp'
siglay_slider =  pnw.IntSlider(name='siglay', start=0, end=len(ds_fvcom.siglay) - 1, value=0)
ds_fvcom[var_name].isel(node=node).interactive.isel(siglay=siglay_slider).hvplot().opts(width=width, height=height)
#print(ds_fvcom[var_name].isel(node=node, time=40, siglay=0))

In [ ]:
# Specify the target datetime range.
start=None; end=None
# Specify siglays where simulated and observed values are compared.
siglays=[3, 11, 19]  # siglay values

for siglay in siglays:
    print(f"siglay={siglay}: {ds_fvcom.z_dfs[:,siglay,node].min().item()} <= z <= {ds_fvcom.z_dfs[:,siglay,node].max().item()} ")
colors = ['blue', 'black', 'red']  # Line colors
plot_config = FvcomPlotConfig(width=8, height=2)
fvcom_ts_plotter = FvcomPlotter(ds_fvcom, plot_config)
save_path = f"fc_ts_{var_name}.png"
fig, ax = plt.subplots(1, 1, figsize=(8, 2)) 
for i, siglay in enumerate(siglays):
    if i < len(siglays) - 1:
        ax=fvcom_ts_plotter.plot_timeseries(var_name, log=False, index=node, k=siglay, start=start, end=end, ax=ax, color=colors[i], alpha=0.7)
    else:
        fvcom_ts_plotter.plot_timeseries(var_name, log=False, index=node, k=siglay, start=start, end=end, ax=ax, color=colors[i], alpha=0.7, save_path=save_path)

In [ ]:
fvcom_z_dfs=[]
for i, siglay in enumerate(siglays):
    fvcom_z_dfs.append(ds_fvcom.z_dfs[:,siglay,node])
    
combined_plot = fvcom_z_dfs[0].hvplot(label=f"siglay={siglays[0]}")
if len(fvcom_z_dfs) >1:
    for i, da in enumerate(fvcom_z_dfs[1:]):
        combined_plot *= da.hvplot(label=f"siglay={siglays[i+1]}")
combined_plot.opts(width=width, height=height, legend_position='right', invert_yaxis=True)

### Interpolate ds_mpos at time-series depth of FVCOM `z_dfs` with interactive plot
`da_mpos_interp_list` is a list of DataArrays.

In [ ]:
hv.renderer('bokeh').theme = 'caliber'
width=800; height=200
size=20; alpha=0.5; marker='o'

mpos_analyzer = MposAnalyzer(ds_mpos)

da_mpos_interp_list = []
for i, siglay in enumerate(siglays):
    da_mpos_interp_list.append(mpos_analyzer.interpolate_timeseries_by_depth(var_name, fvcom_z_dfs[i]))
    # Count the number of valid values (no NaN)
    valid_count = (~np.isnan(da_mpos_interp_list[i])).sum().item()
    if valid_count < 10:
        raise ValueError(f"Number of valid values is too few of {valid_count} at siglay = {siglays[i]}")

combined_plot = da_mpos_interp_list[0].hvplot.line(alpha=alpha, label=f"siglay={siglays[0]}") \
              * da_mpos_interp_list[0].hvplot.scatter(marker=marker, size=size, alpha=alpha)
if len(da_mpos_interp_list) >1:
    for i, da in enumerate(da_mpos_interp_list[1:]):
        combined_plot *= da.hvplot.line(alpha=alpha, label=f"siglay={siglays[i+1]}")
        combined_plot *= da.hvplot.scatter(marker=marker, size=size, alpha=alpha)
combined_plot.opts(width=width, height=height, legend_position='right')
hvplot.save(combined_plot.options(toolbar=None), f"MPOS {var_name} at siglays={siglays}.html")
combined_plot

## Compare FVCOM and MPOS
1. `da_mpos_interp_list` is a list of MPOS DataArrays. `da_fvcom = ds_fvcom[var_name].isel(node=node, siglay=siglay))` is a DataArray of FVCOM. Its list is defined as `da_fvcom_list`.
2. When either of the two DataArrays contains missing values, a `common_mask` is created to mark those time values as missing.
3. Apply `common_mask` to both DataArrays and create DataArray lists of `da_fvcom_cleaned_list` and `da_mpos_interp_cleaned_list`.

In [ ]:
da_fvcom_list = []
for i in range(len(siglays)):
    da = ds_fvcom[var_name].isel(node=node, siglay=siglays[i])
    da_fvcom_list.append(da)
common_mask = []
da_mpos_interp_cleaned_list = []
da_fvcom_cleaned_list = []
for i in range(len(siglays)):
    common_mask.append(~np.isnan(da_mpos_interp_list[i]) & ~np.isnan(da_fvcom_list[i]))
    da_mpos_interp_cleaned_list.append(da_mpos_interp_list[i].where(common_mask[i]))
    da_fvcom_cleaned_list.append(da_fvcom_list[i].where(common_mask[i]))

combined_plot = da_mpos_interp_cleaned_list[0].hvplot.line(alpha=alpha, label=f"siglay={siglays[0]}") \
              * da_fvcom_cleaned_list[0].hvplot.scatter(marker=marker, size=size, alpha=alpha)

if len(siglays) >1:
    for i in range(1, len(siglays)):
        combined_plot *= da_mpos_interp_cleaned_list[i].hvplot.line(alpha=alpha, label=f"siglay={siglays[i]}")
        combined_plot *= da_fvcom_cleaned_list[i].hvplot.scatter(marker=marker, size=size, alpha=alpha)

combined_plot.opts(width=width, height=height, legend_position='right')
hvplot.save(combined_plot.options(toolbar=None), f"FVCOM-MPOS {var_name} at siglays={siglays}.html")
combined_plot


## Evaluate simulation results
Use `evaluate_model_scores` in xfvcom/helpers_utils.py

In [ ]:
sim_list = da_fvcom_cleaned_list
obs_list = da_mpos_interp_cleaned_list
scores = evaluate_model_scores(sim_list, obs_list)
print("Individual Scores:", scores["individual_scores"])
print("Combined Score:", scores["combined_score"])

In [ ]:
sim_list, obs_list = generate_test_data()
sim_list = obs_list
scores = evaluate_model_scores(sim_list, obs_list)
print("Individual Scores:", scores["individual_scores"])
print("Combined Score:", scores["combined_score"])

## Plot example for MPOS using matplotlib

In [ ]:
base_path_mpos = '~/Github/TB-FVCOM/data/MLIT_edited_Mpos/nc_structured/'
# base_path_mpos = '~/Github/TB-FVCOM/data/MLIT_edited_Mpos/nc/'
mpos_loader = MposDataLoader(base_path=base_path)
kemigawa = mpos_loader.load_structured_nc(stn='01kemigawa', start=2010, end=2020, dask=False)
kemigawa

In [ ]:
var_name = 'DO'
mpos_config_1 = MposPlotConfig(figsize=(8, 2), dpi=200, cmap="coolwarm", levels=20)
mpos_plotter_1 = MposPlotter(kemigawa, config=mpos_config_1)
# Create the contour plot
fig, ax, cbar = mpos_plotter_1.plot_contour(
    kemigawa[var_name],
    xlim=('2010-01-01', '2020-01-01'),
    ylim=(0, 10),
    clim=(0, 15),
    stn="Kemigawa",
    rolling=False,
    window=25
)

# Adjust the x-axis and add more annotations
ax.set_xlim(pd.Timestamp('2015-01-01'), pd.Timestamp('2020-01-01'))
ax.set_title("Adjusted DO at Kemigawa")
plt.show()

In [ ]:
var_name = 'DO'
mpos_config_2 = MposPlotConfig(figsize=(8, 2), dpi=200, cmap="coolwarm", levels=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
mpos_plotter_2 = MposPlotter(kemigawa, config=mpos_config_2)
fig, ax, cbar = mpos_plotter_2.plot_mean_annual_cycle(
    kemigawa[var_name], xlim=("2020-01-01", "2022-12-31"),
    ylim=(0, 10), clim=(0, 15), stn="01kemigawa",
    rolling=True, window=24*30+1
)
plt.show()